### Session Connection

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
import configparser

connection_parameters = {
    "account": "<your snowflake account identifier>",
    "user": "<your snowflake username>",
    "password": "<your snowflake password>",
    "role": "<your snowflake role>",
    "warehouse": "<your snowflake warehouse>",  # optional
    "database": "<your snowflake database>",  # optional
    "schema": "<your snowflake schema>" # optional
  }

# it is best practice to not use/save your credentials in a jupyter notebook
# In this case I'm loading my credentials as a dictionary
snowflake_credentials_file = '../snowflake_creds.config'
config = configparser.ConfigParser()
config.read(snowflake_credentials_file)
connection_parameters = dict(config['default'])
session = Session.builder.configs(connection_parameters).create()

### Data Exploration

* Section covers common data exploration steps through Snowpark API
* Code snippets involves reading in a Snowflake table, showcasing different filtering and subsetting options

In [3]:
session.use_database("SNOWPARK_DEFINITIVE_GUIDE")
session.use_schema("MY_SCHEMA")

purchase_history = session.table("PURCHASE_HISTORY")
campaign_info = session.table("CAMPAIGN_INFO")
complain_info = session.table("COMPLAINT_INFO")

In [4]:
purchase_history.show(n=5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
all_rows = purchase_history.collect()
print("frist row: \n ", all_rows[0])

frist row: 
  Row(ID=5524, YEAR_BIRTH=1957, EDUCATION='Graduation', MARITAL_STATUS='Single', INCOME=58138, KIDHOME=0, TEENHOME=0, DT_CUSTOMER=datetime.date(2012, 9, 4), RECENCY=58, MNTWINES=635, MNTFRUITS=88, MNTMEATPRODUCTS=546, MNTFISHPRODUCTS=172, MNTSWEETPRODUCTS=88, MNTGOLDPRODS=88, NUMDEALSPURCHASES=3, NUMWEBPURCHASES=8, NUMCATALOGPURCHASES=10, NUMSTOREPURCHASES=4, NUMWEBVISITSMONTH=7)


##### Count of Rows

In [10]:
purchase_history.count()

2000

##### Displaying Columns

In [11]:
purchase_history.columns

['ID',
 'YEAR_BIRTH',
 'EDUCATION',
 'MARITAL_STATUS',
 'INCOME',
 'KIDHOME',
 'TEENHOME',
 'DT_CUSTOMER',
 'RECENCY',
 'MNTWINES',
 'MNTFRUITS',
 'MNTMEATPRODUCTS',
 'MNTFISHPRODUCTS',
 'MNTSWEETPRODUCTS',
 'MNTGOLDPRODS',
 'NUMDEALSPURCHASES',
 'NUMWEBPURCHASES',
 'NUMCATALOGPURCHASES',
 'NUMSTOREPURCHASES',
 'NUMWEBVISITSMONTH']

##### Filtering -  Row Based Operation

In [16]:
from snowflake.snowpark.functions import col

purchase_history.filter(col("id") == 1)\
.select(col("ID"), col("YEAR_BIRTH"), col("EDUCATION")).show()

-------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |
-------------------------------------
|1     |1961          |Graduation   |
-------------------------------------



In [18]:
purchase_history.filter((col("MARITAL_STATUS") == "Married") & (col("KIDHOME") == 1))\
.select(col("ID"), col("YEAR_BIRTH"), col("EDUCATION"), col("MARITAL_STATUS")).show()

---------------------------------------------------------
|"ID"   |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |
---------------------------------------------------------
|5324   |1981          |PhD          |Married           |
|6177   |1985          |PhD          |Married           |
|1994   |1983          |Graduation   |Married           |
|9736   |1980          |Graduation   |Married           |
|5376   |1979          |Graduation   |Married           |
|2404   |1976          |Graduation   |Married           |
|9422   |1989          |Graduation   |Married           |
|10755  |1976          |2n Cycle     |Married           |
|503    |1985          |Master       |Married           |
|2139   |1975          |Master       |Married           |
---------------------------------------------------------



In [19]:
purchase_history.filter((col("YEAR_BIRTH") >= 1964) & (col("YEAR_BIRTH") <= 1980))\
.select(col("ID"), col("YEAR_BIRTH"), col("EDUCATION"), col("MARITAL_STATUS")).show()

--------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |
--------------------------------------------------------
|4141  |1965          |Graduation   |Together          |
|7446  |1967          |Master       |Together          |
|965   |1971          |Graduation   |Divorced          |
|4855  |1974          |PhD          |Together          |
|387   |1976          |Basic        |Married           |
|9736  |1980          |Graduation   |Married           |
|5376  |1979          |Graduation   |Married           |
|7892  |1969          |Graduation   |Single            |
|2404  |1976          |Graduation   |Married           |
|1966  |1965          |PhD          |Married           |
--------------------------------------------------------



##### Select -  Subsetting Operation

In [11]:
purchase_history.select(col("ID"), col("YEAR_BIRTH"), col("EDUCATION")).show()

-------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |
-------------------------------------
|5524  |1957          |Graduation   |
|2174  |1954          |Graduation   |
|4141  |1965          |Graduation   |
|6182  |1984          |Graduation   |
|5324  |1981          |PhD          |
|7446  |1967          |Master       |
|965   |1971          |Graduation   |
|6177  |1985          |PhD          |
|4855  |1974          |PhD          |
|5899  |1950          |PhD          |
-------------------------------------



In [12]:
#### Columns Can Also Referenced as Follows
purchase_history.select(purchase_history.ID, purchase_history.Year_BIRTH, 
                        purchase_history.EDUCATION).show() # Only if column names contains no space

purchase_history.select(purchase_history["ID"], purchase_history["Year_BIRTH"], 
                        purchase_history["EDUCATION"]).show()

purchase_history.select("ID","YEAR_BIRTH","EDUCATION").show()

-------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |
-------------------------------------
|5524  |1957          |Graduation   |
|2174  |1954          |Graduation   |
|4141  |1965          |Graduation   |
|6182  |1984          |Graduation   |
|5324  |1981          |PhD          |
|7446  |1967          |Master       |
|965   |1971          |Graduation   |
|6177  |1985          |PhD          |
|4855  |1974          |PhD          |
|5899  |1950          |PhD          |
-------------------------------------

-------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |
-------------------------------------
|5524  |1957          |Graduation   |
|2174  |1954          |Graduation   |
|4141  |1965          |Graduation   |
|6182  |1984          |Graduation   |
|5324  |1981          |PhD          |
|7446  |1967          |Master       |
|965   |1971          |Graduation   |
|6177  |1985          |PhD          |
|4855  |1974          |PhD          |
|5899  |195

### Data Transformation

* Section explains various data transformation steps thorughs joins and unions
* Code snippets delineates use of joins and union with focus on usage of required suffix
* Commented code blocks provide additional information on required column selection methodlogy

##### Joins

In [26]:
purchase_campaign = purchase_history.join(campaign_info, purchase_history.ID == campaign_info.ID\
                            ,lsuffix="_left", rsuffix="_right")

purchase_campaign = purchase_campaign.drop("ID_RIGHT")
purchase_campaign.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID_LEFT"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"RESPONSE"  |
--------------------------------------------------------------------------------------------------------------------------

In [27]:
final_combined = purchase_campaign.join(complain_info, purchase_campaign["ID_LEFT"] == complain_info.ID)
final_combined = final_combined.drop("ID_LEFT")
final_combined.show()


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"RESPONSE"  |"ID"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |
------------------------------------------

In [28]:
final_combined.write.save_as_table("MARKETING_DATA")

In [126]:
## Join Column Name - Same on Right & Left Table
# campaign_info = session.table("CAMPAIGN_INFO")
# purchase_history.join(campaign_info, "ID").show()

In [127]:
### Avoiding Random Suffix
# campaign_info = session.table("CAMPAIGN_INFO")
# purchase_history.join(campaign_info, purchase_history.ID == campaign_info.ID,lsuffix="_left", rsuffix="_right").show()


In [128]:
## Selecting Required Columns

# campaign_info = session.table("CAMPAIGN_INFO")
# purchase_history.join(campaign_info, purchase_history.ID == campaign_info.ID,
#         lsuffix="_left", rsuffix="_right")\
#         .select("ID_LEFT","YEAR_BIRTH","EDUCATION","ACCEPTEDCMP3","ACCEPTEDCMP4")\
#         .show()

##### UNION

In [29]:
marketing_additional = session.table("MARKETING_ADDITIONAL")
marketing_additional.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |"RESPONSE"  |
----------------------------------------

In [30]:
print("No of rows in MARKETING_ADDITIONAL table: ",marketing_additional.count())
print("No of rows in PURCHASE_HISTORY table: ",final_combined.count())

final_appended = final_combined.union_by_name(marketing_additional)

print("No of rows in UPDATED table: ",final_appended.count())
final_appended.show()

No of rows in MARKETING_ADDITIONAL table:  240
No of rows in PURCHASE_HISTORY table:  2000
No of rows in UPDATED table:  2240
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  

In [32]:
final_appended.write.save_as_table("MARKETING_DATA", mode="overwrite")